### Importación de librerías a utilizar

In [7]:
import numpy as np
import matplotlib.pyplot as plt

### Función de polling table: Tabla de sondeo para ONUs

In [8]:
def create_polling_table(onu_no, onu_olt_dist, grant_size,link_speed):
    c = 2*1e5
    # Inicialización de tabla con sondeos
    polling_table = np.zeros((onu_no,4))
    
    #indice de ONU
    polling_table[:, 0] = np.arange(1, onu_no + 1)

    # Round Time Trip (RTT)
    polling_table[:, 1] = 2 * onu_olt_dist / c

    # Tiempo de Transmisión
    polling_table[:, 2] = grant_size * 8 / link_speed

    # Columna 4: Tiempo total (RTT + tiempo de transmisión).
    polling_table[:, 3] = polling_table[:, 1] + polling_table[:, 2]

    return polling_table

### Función pkt_arr_times: Tiempos de llegada de paquetes

In [9]:
def pkt_arr_times(type, par1, par2=None, max_pkts=None):
    if type == 'deterministic':
        avg_arr_time = par1
        pkt_arr_time = avg_arr_time * np.ones(max_pkts)

    elif type == 'exponential':
        avg_arr_time = par1
        pkt_arr_time = np.random.exponential(avg_arr_time, max_pkts)

    elif type == 'uniform':
        min_arr_time = par1
        max_arr_time = par2
        pkt_arr_time = min_arr_time + (max_arr_time - min_arr_time) * np.random.rand(max_pkts)

    elif type == 'GP':  # Generalized Pareto distribution
        k = par1
        sigma = par2
        theta = 0  # minimum value for time is 0
        pkt_arr_time = np.random.default_rng().pareto(k, max_pkts) * sigma + theta

    elif type == 'gamma':
        a = par1
        b = par2
        pkt_arr_time = np.random.gamma(a, b, max_pkts)

    elif type == 'Gaussian':
        mean = par1
        variance = par2
        pkt_arr_time = mean + variance * np.abs(np.random.randn(max_pkts))

    else:
        raise ValueError("Unsupported distribution type")

    return pkt_arr_time


### Función plot_results: Gráfica de resultados según ciclo de trabajo 

In [ ]:
def plot_results(Lambda, polling_cycles, avg_pkt_delay_all, avg_pkt_trx_delay_all):
    # ============================ figure 1 ============================
    plt.figure()
    plt.plot(Lambda, polling_cycles, '-o', linewidth=2)
    plt.grid(True)
    plt.box(True)
    plt.xlim([0, 1])
    plt.ylim([0, np.inf])
    plt.xlabel('Effective network load $(\\rho)$', fontweight='bold', fontsize=12)
    plt.ylabel('Number of polling cycles', fontweight='bold', fontsize=12)
    plt.legend(['Limited service'], loc='northeast', fontsize=10)
    plt.title('Polling Cycles vs Effective Load')
    plt.show()

    # ============================ figure 2 ============================
    plt.figure()
    plt.semilogy(Lambda, avg_pkt_delay_all, '-o', linewidth=2)
    plt.grid(True)
    plt.box(True)
    plt.xlim([0, 1])
    plt.ylim([1e-4, 1e-1])
    plt.xlabel('Effective network load $(\\rho)$', fontweight='bold', fontsize=12)
    plt.ylabel('Average packet latency at ONU (sec)', fontweight='bold', fontsize=12)
    plt.legend(['Limited service'], loc='northeast', fontsize=10)
    plt.title('Average Packet Latency vs Effective Load')
    plt.show()

    # ============================ figure 3 ============================
    plt.figure()
    plt.semilogy(Lambda, avg_pkt_trx_delay_all, '-o', linewidth=2)
    plt.grid(True)
    plt.box(True)
    plt.xlim([0, 1])
    plt.ylim([1e-4, 1e-1])
    plt.xlabel('Effective network load $(\\rho)$', fontweight='bold', fontsize=12)
    plt.ylabel('Average end-to-end packet latency (sec)', fontweight='bold', fontsize=12)
    plt.legend(['Limited service'], loc='northeast', fontsize=10)
    plt.title('End-to-End Packet Latency vs Effective Load')
    plt.show()